## imports

In [ ]:
!pip install -q datasets transformers jsonlines

In [ ]:
from tqdm import tqdm
from joblib import dump
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
import jsonlines
import torch

## load dataset

In [ ]:
labels = []
texts = []

ds = load_dataset("K-Monitor/kmdb_base")
for n in ds['train']:
    labels.append(n['category'])
    texts.append(n['text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/58750 [00:00<?, ? examples/s]

## load model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = BertForSequenceClassification.from_pretrained('boapps/kmdb_classification_model').to(device)
tokenizer = BertTokenizer.from_pretrained('SZTAKI-HLT/hubert-base-cc')

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/272k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

In [ ]:
def get_bert_embeddings(texts):
    model.eval()
    with torch.no_grad():
        embeddings = []
        for text in tqdm(texts):
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            output = model(**inputs, output_hidden_states=True)
            cls_embedding = output.hidden_states[-1][:, 0, :]
            embeddings.append(cls_embedding.squeeze().to('cpu').numpy())
    return embeddings

embeddings = get_bert_embeddings(texts)

 17%|█▋        | 9830/58750 [3:04:44<16:13:50,  1.19s/it]

In [ ]:
# Prepare train and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.25, random_state=42)

## train

In [ ]:
# Initialize and train the SVM
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

## evaluate

In [ ]:
predictions = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

## upload

In [ ]:
dump(svm_classifier, 'svm_classifier_category.joblib')